In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using HybridRobotDynamics
using HiLQR

In [ ]:
"""
Solver Setup
"""

# Bouncing ball with thrust model
system = bouncing_ball()

# Stage and terminal costs
Q = 1e-6 * diagm([1.0, 1.0, 0.0, 0.0])
R = 1e-4 * I(system.nu)
Qf = 1e6 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.02
params = HiLQR.Parameters(system, stage, terminal, rk4, N, Δt)

# Reference trajectory and initial conditions
xref = [10.0; 2.0; 0.0; 0.0]
uref = zeros(system.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]
params.x0 = [0.0, 2.0, 10.0, -10.0]
params.mI = :flight

nothing

In [ ]:
"""
Solve using HiLQR
"""

# Solve
sol = HiLQR.Solution(params)
cache = HiLQR.Cache(params)
@time HiLQR.solve!(sol, cache, params; multishoot=true, αmax=1.0)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, system.nx, (1,2), xs)

nothing